In [1]:
!conda install nltk -y
!pip install sklearn
!pip install keytotext
!pip install pandas
!pip install gensim
!pip install spacy
!pip install numpy
!pip install rake_nltk
!python3 -m spacy download en_core_web_sm

#import extractkeys
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
from keytotext import trainer, pipeline, make_dataset
import pandas as pd
import json
import os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from nltk.tokenize.treebank import TreebankWordDetokenizer
from rake_nltk import Rake
import spacy
from spacy import displacy
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')
os.environ["TOKENIZERS_PARALLELISM"] = "True"

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - nltk


The following NEW packages will be INSTALLED:

  click              pkgs/main/noarch::click-8.0.3-pyhd3eb1b0_0
  joblib             pkgs/main/noarch::joblib-1.1.0-pyhd3eb1b0_0
  nltk               pkgs/main/noarch::nltk-3.6.5-pyhd3eb1b0_0
  regex              pkgs/main/linux-64::regex-2021.8.3-py38h7f8727e_0

The following packages will be UPDATED:

  ca-certificates                      2021.5.25-h06a4308_1 --> 2021.10.26-h06a4308_2
  certifi                          2021.5.30-py38h06a4308_0 --> 2021.10.8-py38h06a4308_0
  conda                               4.10.1-py38h06a4308_1 --> 4.10.3-py38h06a4308_0
  openssl                                 1.1.1k-h27cfd23_0 --> 1.1.1l-h7f8727e_0


Preparing transaction: done
Verifying transaction: failed

EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
2021-12-05 18:23:26.309712: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.9 MB 528 kB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Global seed set to 42
[nltk_data] Downloading package punkt to /home/jwinchell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jwinchell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def head(sentence, verbose=False):
    nlp = spacy.load("en_core_web_sm")
    detokenize = TreebankWordDetokenizer().detokenize
    sentence = detokenize(sentence)
    doc = nlp(sentence)
    if verbose:
        print("\n{:<15} | {:<8} | {:<15} | {:<20}".format('Token', 'Relation', 'Head', 'Children'))
        print("-" * 70)
        for token in doc:
            # Print the token, dependency nature, head and all dependents of the token
            print("{:<15} | {:<8} | {:<15} | {:<20}"
                  .format(str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))
        # Use displayCy to visualize the dependency
        displacy.render(doc, style='dep', options={'distance': 120})
        print()
    for token in doc:
        # find verb head
        if str(token.dep_) == "ROOT":
            children = []
            for child in token.children:
                # find subj nouns
                if child.dep_ in ("nsubj", "dobj", "compound"):
                    children.append(child)
            return [token] + children


def rake(sent):
    rake = Rake()
    if not isinstance(sent, str):
        detokenize = TreebankWordDetokenizer().detokenize
        sent = detokenize(sent)
    #print(sent)
    kw = rake.extract_keywords_from_text(sent)
#     print(f"kw:{kw}")
    ranked_phrases = rake.get_ranked_phrases()
#     print(f"ranked:{ranked_phrases}")
    return ranked_phrases


def label(subject_lines):
    labeled = {
        "text": [],
        "keywords": []
    }
    # extract keys from subject lines as their labels
    length = len(subject_lines)
    i = 0
    for line in subject_lines:
        if len(line) <= 1:
            length -= 1
            continue
        try:
            phrase = rake(line)[0]
        except IndexError as e:
            print(f"Error: {line} -- {phrase}\t{e}")
            length -= 1
            continue
        tok_phrase = word_tokenize(phrase)
        if len(tok_phrase) <= 1:
            length -= 1
            continue
        x = head(tok_phrase, False)
        if len(x) < 3:
            length -= 1
            continue
        if not x:
            length -= 1
            continue
        keywords = ''
        for token in x:
            keywords += str(token) + ' '
        # print(f"{line} --> {keywords}")
        detokenize = TreebankWordDetokenizer().detokenize
        labeled["text"].append(detokenize(line))
        labeled["keywords"].append(keywords)
        i += 1
        print(f"{i} / {length}         ", end="\r")
    df = pd.DataFrame(labeled)
    return df


def train(labeled_data):
    train, test = train_test_split(labeled_data, shuffle=True)
    model = trainer()
    model.from_pretrained()
    model.train(train_df=train, test_df=test, use_gpu=False)
    return model


f = open('opened_data.json', )
data = json.load(f)['emails']
subject_lines = []
print("Starting...")
for email in data:
    try:
        if email[0][0] and len(email[0][0]) > 4:
            subject_lines.append(email[0][0])
    except IndexError as e:
        pass
#         print(f"{e}\n{email}")
#         exit(1)
if not os.path.isfile('labelled_data.csv'):
    print("Labelling has started...")
    df = label(subject_lines)
    df.to_csv('labelled_data.csv')
else:
    df = pd.read_csv('labelled_data.csv')
# df = 'm'
print("Training has started...")
model = train(df)
model.save_model(model_dir="model")

Starting...
Training has started...


/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=5)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1579: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as:


  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 19. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 22. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
Global seed set to 42
/home/jwin

...

/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 15. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 24. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 26. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/jwinchell/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infe

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [12]:
model.predict(["paid", "fall", "applications"], use_gpu=False)

'Fall 2018 paid applications now available.'